## Load data

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
random_state = 12041500

In [2]:
df_train = pd.read_json("./data/synthetic_data_CTGANSynthesizer.json")
df_test = pd.read_json("./data/testset.json")

df_train.drop(columns=['fnlwgt'], inplace=True)
df_test.drop(columns=['fnlwgt'], inplace=True)

ratio_features = ["age", "capital-gain", "capital-loss", "hours-per-week"]
ordinal_features = ["education-num"]
nominal_features = ['workclass', 'marital-status', 'occupation', 'relationship', 'race', 'sex']
target = 'income'

## Train baseline

In [3]:
from sklearn.linear_model import LogisticRegression
from utils import create_model, train_and_evaluate, describe_model

In [4]:
clf = LogisticRegression(max_iter=1000, random_state=random_state)

In [5]:
## Train baseline
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_test, target, drop_na=True)
metrics = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.836
Precision      0.782
Recall         0.746
F1             0.761


## Fairness Evaluation

In [6]:
from utils import split_data
from utils_fairness import search_bias, calc_fairness_score, explain_detected_bias

In [7]:
X_train, y_train = split_data(df_train, target, drop_na=True)

In [8]:
model = create_model(clf, nominal_features)
model.fit(X_train, y_train)
probs = pd.Series(model.predict_proba(X_train)[:, 1]) # we select for target label

In [9]:
privileged_subset, _ = search_bias(X_train, y_train, probs, 1, penalty=1)

In [10]:
print(privileged_subset)

({'capital-gain': [0, 1, 4, 5, 6]}, 226.8525)


In [11]:
calc_fairness_score(df_train, privileged_subset[0].keys(), target, verbose=True)

Sensitive Attributes: ['capital-gain']

                         Group Distance  Proportion  Counts   P-Value
capital-gain [37.00, 16383.00]    0.434    0.095449    3563  0.00e+00
    capital-gain [-0.00, 5.00]   -0.137    0.517078   19302 1.48e-323
   capital-gain [23.00, 37.00]    0.253    0.097458    3638 3.41e-256
   capital-gain [15.00, 23.00]    0.087    0.106352    3970  1.82e-38
    capital-gain [5.00, 10.00]   -0.047    0.097645    3645  2.44e-13

Weighted Mean Statistical Distance: 0.1648489023109496


In [12]:
explain_detected_bias(df_train, probs, target, privileged_subset[0])

Our detected privileged group has a size of 19340, we observe 0.0639 as the average probability of earning >50k, but our model predicts 0.2058


## Fairness Metrics

In [13]:
from utils_fairness import transform_to_bias_dataset, describe_fairness, scan_and_calculate_fairness, plot_fairness_metrics

In [14]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

1744 Na rows removed!
202 Na rows removed!


In [15]:
model = create_model(clf, nominal_features)
y_test, y_pred = train_and_evaluate(model, df_train, df_train, target, drop_na=True)
metrics = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.220
average_odds_difference         0.182
equal_opportunity_difference    0.297
disparate_impact                5.949
theil_index                     0.114


In [16]:
df_fairness_metrics, priviliged_subsets = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
), {}

for i in [1e-17, 1e-10, 0.001, 0.01, 0.1, 0, 1, 5, 10, 25, 50, 100]:
    metrics, priv = scan_and_calculate_fairness(model, df_train, target, i)    
    df_fairness_metrics.loc[f"{i}"] = metrics.values()
    priviliged_subsets[f"{i}"] = priv

In [17]:
df_fairness_metrics

,statistical_parity_difference,average_abs_odds_difference,equal_opportunity_difference,disparate_impact,theil_index
1e-17,0.157790,0.301264,0.548028,80.920614,0.113518
1e-10,0.176903,0.143317,0.245121,4.778996,0.113518
0.001,-0.072405,-0.190094,-0.251483,0.678597,0.113518
0.01,0.106165,0.276341,0.548028,3.123298,0.113518
0.1,0.164673,0.148689,0.262768,4.440570,0.113518
0,0.157790,0.301264,0.548028,80.920614,0.113518
1,0.219782,0.182051,0.297375,5.948852,0.113518
5,0.208665,0.180456,0.297944,6.022775,0.113518
10,0.201300,0.189152,0.317138,6.455366,0.113518
25,0.201300,0.189152,0.317138,6.455366,0.113518


In [18]:
df_fairness_metrics.to_json("./results/fairness_metrics_synthetic.json")

## Mitigation

In [18]:
from tqdm import tqdm
from sklearn import clone
from aif360.algorithms.preprocessing import Reweighing
from utils_fairness import create_aif360_standardDataset, compute_metrics, reweight_mitigation
from utils import plot_metrics

### Reweighting

In [19]:
# create (un)privileged groups
privileged_groups = [{key: 1 for key in list(privileged_subset[0].keys())}]
unprivileged_groups = [{key: 0 for key in list(privileged_subset[0].keys())}]

In [20]:
# convert dataset
train_dataset = create_aif360_standardDataset(df_train.dropna(), nominal_features, target, 1, list(privileged_subset[0].keys()), list(privileged_subset[0].values()))
test_dataset = create_aif360_standardDataset(df_test.dropna(), nominal_features, target, 1, list(privileged_subset[0].keys()), list(privileged_subset[0].values()))

In [21]:
RW = Reweighing(unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
RW.fit(train_dataset)

train_dataset_reweight = RW.transform(train_dataset)

#### Baseline

In [22]:
model = clone(clf)
model.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.826
Precision      0.801
Recall         0.680
F1             0.710


In [23]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.294
average_odds_difference         0.326
equal_opportunity_difference    0.533
disparate_impact                19.095
theil_index                     0.108


#### Mitigated Model

In [24]:
# mitigated model
model = clone(clf)
model.fit(train_dataset_reweight.features, train_dataset_reweight.labels.ravel(), sample_weight=train_dataset_reweight.instance_weights)
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.815
Precision      0.747
Recall         0.754
F1             0.750


In [25]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.107
average_odds_difference         -0.002
equal_opportunity_difference    0.002
disparate_impact                2.234
theil_index                     0.134


#### Bias detection & mitigation until bias free

In [26]:
df_fairness_metrics = pd.DataFrame(
    columns=[
        "statistical_parity_difference",
        "average_abs_odds_difference",
        "equal_opportunity_difference",
        "disparate_impact",
        "theil_index",
    ]
)

In [27]:
df_metrics = pd.DataFrame(
    columns=['acc', 'prec', 'rec', 'f1']
)

In [28]:
weights = None

weights_hist = [weights]
max_iter = 25
for i in tqdm(range(max_iter)):
    X_train, y_train = split_data(df_train, target, True)
    X_test, y_test = split_data(df_test, target, True)
    weights, model_metrics, fair_metrics = reweight_mitigation(clf, nominal_features, target, X_train, y_train, X_test, y_test, penalty=1, sample_weights=weights)
    if model_metrics is None and fair_metrics is None and weights is None:
        break

    df_metrics.loc[f"mitigation_{i}"] = model_metrics.values()
    df_fairness_metrics.loc[f"mitigation_{i}"] = fair_metrics.values()
    weights_hist.append(weights)

100%|██████████| 25/25 [12:12<00:00, 29.30s/it]


In [29]:
fairness_metrics_json = df_fairness_metrics.to_json(path_or_buf="./results/fairness_metrics_25_iterations_synthetic.json")
metrics_json = df_metrics.to_json(path_or_buf="./results/utility_metrics_25_iterations_synthetic.json")

### Fair Learning

In [30]:
from utils import prepare_data_fair_learning, plot_fairlearning_results
from utils_fairness import get_fair_learning_scoring
from sklearn.model_selection import GridSearchCV
from aif360.sklearn.preprocessing import LearnedFairRepresentations

In [31]:
df_train_bias = transform_to_bias_dataset(df_train, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)
df_test_bias = transform_to_bias_dataset(df_test, list(privileged_subset[0].keys()), list(privileged_subset[0].values()), verbose=True)

X_train, y_train, X_test, y_test = prepare_data_fair_learning(df_train_bias, df_test_bias, nominal_features, target)

1744 Na rows removed!
202 Na rows removed!


In [32]:
X_train['capital-gain'] = X_train.index
X_test['capital-gain'] = X_test.index

In [33]:
max_delta = get_fair_learning_scoring(list(privileged_subset[0].keys()))

In [34]:
lfr = LearnedFairRepresentations(
    list(privileged_subset[0].keys()),
    n_prototypes=20,
    max_iter=50,
    random_state=random_state,
)

In [35]:
params = {
    "reconstruct_weight": [1e-2, 1e-3, 1e-4],
    "target_weight": [100, 1000],
    "fairness_weight": [0, 100, 1000],
}

In [36]:
grid = GridSearchCV(lfr, params, scoring=max_delta, cv=3, n_jobs=-1).fit(
    X_train, y_train, priv_group=(1,) * len(list(privileged_subset[0].keys()))
)
res = pd.DataFrame(grid.cv_results_)

/Users/maxkleinegger/Documents/TUWien/2023WS/BA/.ba-thesis/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')
/Users/maxkleinegger/Documents/TUWien/2023WS/BA/.ba-thesis/lib/python3.10/site-packages/torch/_functorch/deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/m

#### Baseline

In [37]:
model = clone(clf)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

_ = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.827
Precision      0.802
Recall         0.682
F1             0.711


In [38]:
y_pred = model.predict(X_train)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.293
average_odds_difference         0.325
equal_opportunity_difference    0.531
disparate_impact                18.818
theil_index                     0.108


#### Using Grid itself

In [39]:
_ = describe_model(y_test, grid.predict(X_test), verbose=True)

Metric         Value               
Accuracy       0.759
Precision      0.627
Recall         0.522
F1             0.488


In [40]:
_ = describe_fairness(y_train, grid.predict(X_train), list(privileged_subset[0].keys()) ,verbose=True)

Metric                          Value               
statistical_parity_difference   0.019
average_odds_difference         0.019
equal_opportunity_difference    0.026
disparate_impact                10.277
theil_index                     0.223


#### Transforming data

In [41]:
model = clone(clf)
model.fit(grid.transform(X_train), y_train)
y_pred = model.predict(X_test)

_ = describe_model(y_test, y_pred, verbose=True)

Metric         Value               
Accuracy       0.450
Precision      0.554
Recall         0.564
F1             0.447


In [42]:
_ = describe_fairness(y_train, model.predict(X_train), list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.084
average_odds_difference         0.064
equal_opportunity_difference    0.054
disparate_impact                1.116
theil_index                     0.085


##### Also Transforming test data

In [43]:
_ = describe_model(y_test, model.predict(grid.transform(X_test)), verbose=True)

Metric         Value               
Accuracy       0.766
Precision      0.662
Recall         0.587
F1             0.596


In [44]:
_ = describe_fairness(y_train, model.predict(grid.transform(X_train)), list(privileged_subset[0].keys()) ,verbose=True)

Metric                          Value               
statistical_parity_difference   0.123
average_odds_difference         0.100
equal_opportunity_difference    0.141
disparate_impact                3.928
theil_index                     0.177


Multiple Runs cannot be done, because of the dataset already needs to be one hot encoded for the mitigation, making the `search_bias` function unnecessary and returning not viable resolutions.

### Fair Learning

In [46]:
%matplotlib inline
# Load all necessary packages

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult
from aif360.algorithms.preprocessing.lfr import LFR

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import numpy as np
from utils_fairness import create_aif360_standardDataset

In [47]:
# create (un)privileged groups
privileged_groups = [{key: 1 for key in list(privileged_subset[0].keys())}]
unprivileged_groups = [{key: 0 for key in list(privileged_subset[0].keys())}]

In [48]:
TR = LFR(unprivileged_groups=unprivileged_groups,
         privileged_groups=privileged_groups,
         k=10, Ax=0.01, Ay=1000, Az=0,
         verbose=1,
         seed=random_state
)

TR = TR.fit(train_dataset, maxiter=5000, maxfun=1000)

step: 0, loss: 605.8719525156314, L_x: 4089.307287970777,  L_y: 0.5649788796359236,  L_z: 0.000442105866442874
step: 250, loss: 605.8719527432613, L_x: 4089.307287989528,  L_y: 0.564978879863366,  L_z: 0.000442105806862049
step: 500, loss: 605.8719475746043, L_x: 4089.307287967263,  L_y: 0.5649788746949316,  L_z: 0.0004421057161778499
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          500     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.05872D+02    |proj g|=  2.78235D+01
step: 750, loss: 559.2910323520119, L_x: 4088.5294761775235,  L_y: 0.5184057375902368,  L_z: 0.0004318659186486676
step: 1000, loss: 559.291034705393, L_x: 4088.5294761868945,  L_y: 0.518405739943524,  L_z: 0.00043186608255237833

At iterate    1    f=  5.59291D+02    |proj g|=  1.09570D+01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored d

In [49]:
# Transform training data and align features
train_dataset_lfr = TR.transform(train_dataset)

In [50]:
cnt = {}
for i in train_dataset_lfr.labels.ravel():
    cnt[i] = cnt.get(i, 0) + 1

cnt

{0.0: 37329}

#### Baseline

In [51]:
model = clone(clf)
model.fit(train_dataset.features, train_dataset.labels.ravel())
y_pred = model.predict(test_dataset.features)

_ = describe_model(test_dataset.labels.ravel(), y_pred, verbose=True)

Metric         Value               
Accuracy       0.826
Precision      0.801
Recall         0.680
F1             0.710


In [52]:
y_pred = model.predict(train_dataset.features)
_ = describe_fairness(df_train_bias[target], y_pred, list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.294
average_odds_difference         0.326
equal_opportunity_difference    0.533
disparate_impact                19.095
theil_index                     0.108


#### Mitigation

In [53]:
_ = describe_model(df_test_bias[target], [0]*len(df_test_bias[target]), verbose=True)

Metric         Value               
Accuracy       0.759
Precision      0.380
Recall         0.500
F1             0.432


In [54]:
_ = describe_fairness(df_train_bias[target], np.array([0]*len(df_train_bias)), list(privileged_subset[0].keys()), verbose=True)

Metric                          Value               
statistical_parity_difference   0.000
average_odds_difference         0.000
equal_opportunity_difference    0.000
disparate_impact                0.000
theil_index                     0.229


### Disparate Impact Remover

In [55]:
import numpy as np
from tqdm import tqdm

from aif360.algorithms.preprocessing import DisparateImpactRemover

In [56]:
index = train_dataset.feature_names.index('capital-gain')

In [57]:
fairness_metrics = []
utility_metrics = []
for level in tqdm(np.linspace(0, 1, 10)):
    di = DisparateImpactRemover(repair_level=level)
    train_repd = di.fit_transform(train_dataset)
    test_repd = di.fit_transform(test_dataset)
    
    X_tr = train_repd.features # np.delete(train_repd.features, index, axis=1)
    X_te = test_repd.features #np.delete(test_repd.features, index, axis=1)
    y_tr = train_repd.labels.ravel()
    
    model = clone(clf)
    model.fit(X_tr, y_tr)

    utility_metrics.append(describe_model(test_repd.labels.ravel(), model.predict(X_te), verbose=False))
    fairness_metrics.append(describe_fairness(df_train_bias[target], model.predict(X_tr), list(privileged_subset[0].keys()), verbose=False))


100%|██████████| 10/10 [01:20<00:00,  8.08s/it]


In [58]:
fairness_metrics

[{'statistical_parity_difference': 0.2940092562267429,
  'average_odds_difference': 0.3263848052479457,
  'equal_opportunity_difference': 0.5325510761059932,
  'disparate_impact': 19.095289689901932,
  'theil_index': 0.1077590833288912},
 {'statistical_parity_difference': 0.2926759922159685,
  'average_odds_difference': 0.32400351074853,
  'equal_opportunity_difference': 0.5288239686829213,
  'disparate_impact': 18.60250658070242,
  'theil_index': 0.10785454947161889},
 {'statistical_parity_difference': 0.29601781229946794,
  'average_odds_difference': 0.3273713007937284,
  'equal_opportunity_difference': 0.5331354249337793,
  'disparate_impact': 18.92023897146681,
  'theil_index': 0.10706475179598487},
 {'statistical_parity_difference': 0.2930458985007494,
  'average_odds_difference': 0.324333673654631,
  'equal_opportunity_difference': 0.5289796834135347,
  'disparate_impact': 18.567530746486483,
  'theil_index': 0.10789597007646609},
 {'statistical_parity_difference': 0.293315544498

In [59]:
max_index, max_disparate_impact_row = max(enumerate(fairness_metrics), key=lambda x: x[1]['disparate_impact'])
max_disparate_impact_row

{'statistical_parity_difference': 0.2940092562267429,
 'average_odds_difference': 0.3263848052479457,
 'equal_opportunity_difference': 0.5325510761059932,
 'disparate_impact': 19.095289689901932,
 'theil_index': 0.1077590833288912}

In [60]:
utility_metrics[max_index]

{'Accuracy': 0.8263823560154698,
 'Precision': 0.8009893116559623,
 'Recall': 0.680496775358145,
 'F1': 0.7095284659429854}